<a href="https://colab.research.google.com/github/q890003/ML_logistic-regression-EM/blob/master/ML_hw4_EM_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import dill
import numpy as np
import math
from itertools import product 
from struct import unpack 
import numba as nb

def get_img(img_file_path):
    with open(img_file_path, "rb") as img_file:
        magic, num_imgs, rows, cols = unpack(">IIII", img_file.read(16))

        imgs_mtx = np.zeros((num_imgs, rows, cols), dtype=int)
        for img in range(num_imgs):
            for (row, col) in product(range(rows), range(cols)):
                tmp = unpack(">B", img_file.read(1))
                tmp = int(str(tmp[0]))
                imgs_mtx[img][row,col] = tmp
    return imgs_mtx, num_imgs

def get_label(lbl_file_path):
    with open(lbl_file_path, "rb") as label_file:
        magic, num_itms = unpack(">II", label_file.read(8))
        lbls_mtx = np.zeros((num_itms), dtype=int)
        for img in range(num_itms):
            label = unpack(">B", label_file.read(1))
            label = int(str(label[0]))
            lbls_mtx[img] = label
    return lbls_mtx

label_types = 10
gray_bins = 2
rows = 28
cols = 28

train_img_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-images.idx3-ubyte"
train_label_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-labels.idx1-ubyte"
train_img_mtx, train_num_imgs = get_img(train_img_pth)
train_label_mtx = get_label(train_label_pth)

labels_count = np.zeros((label_types), dtype=float)
for img in range(train_num_imgs):
    lbl = train_label_mtx[img]
    labels_count[lbl] += 1
    train_img_mtx[img] = train_img_mtx[img] // 128
  

In [0]:
@nb.jit
def estep(lamda, mu, train_img_mtx):
    weight = np.zeros((60000, 10), dtype=np.float64)
    for img in range(60000):
        for label in range(10):
            weight[img][label] = np.prod(2* (mu[label]*train_img_mtx[img] + (1- mu[label])*(1 - train_img_mtx[img]) ))
        weight[img] = weight[img]*lamda
        total = np.sum(weight[img])
        weight[img] /= total
    return weight

@nb.jit
def mstep(weight, train_img_mtx):
    mu_new = np.zeros((10,28,28), dtype=np.float64)
    for label in range(10):
        for img in range(60000):
            mu_new[label] += weight[img][label] * train_img_mtx[img]  

    sigma_w = np.sum(weight, axis=0) 
    
    for label in range(10): 
        mu_new[label] /= sigma_w[label] 
    #recalculate lamda of each cluster. 
    lamda = sigma_w/60000
    return mu_new, lamda
@nb.jit
def map_class(train_label_mtx, weight, mu):
    count = np.zeros((10, 10), dtype=np.float64)
    map_idx = np.zeros((10), dtype=np.int32)
    idx = 0
    for img in range(60000):
        count[train_label_mtx[img], np.argmax(weight[img]) ] += 1
    for label in range(10):
        idx = np.argmax(count[label])
        map_idx[label] = idx
    return map_idx
@nb.jit
def print_imag(map_idx, mu, mu_new, iteration):
    imagination_number = np.zeros((10, 28, 28), dtype=np.float64)
    imagination_number = mu_new.copy()
    imagination_number[imagination_number > 0.5] = 1
    imagination_number[imagination_number <= 0.5] = 0
    print("------------------------------------------------------------")
    for label in range(10):
        print("labeled class ",label,":")
        for (r,c) in product(range(28), range(28)):
            print(int(imagination_number[map_idx[label]][r][c]),end=" ")
            if c == 27:
                print("")
    print("No. of Iteration:{it}, Difference: {err}\n".format(it = iteration, err = np.sum(abs(mu_new - mu))))
    print("------------------------------------------------------------")

#For record
lamda_1 = np.empty(label_types)
mu_1 = np.empty((label_types,28,28))

lamda_9 = np.empty((label_types))
mu_9 = np.empty((label_types,28,28))

# init
lamda = np.full((label_types), 1/10)  #prob of a nmber under numbers 0~9
mu = np.random.rand(label_types,28,28)  #each cluster has 28*28 dim mu.
weight = np.zeros((train_num_imgs, label_types), dtype=float)  #record each cluster's weight to each data.
converge_count = 0
for it in range(100):
    # Expectation step:
    weight = estep(lamda, mu, train_img_mtx)

    # Maximization step:
    # According to each cluster, recalculate mu of a cluster. 
    mu_new, lamda = mstep(weight, train_img_mtx)
    
    if it == 1:
        mu_1 = mu_new.copy()
        lamda_1 = lamda.copy()
    if it == 9:
        mu_9 = mu_new.copy()
        lamda_9 = lamda.copy()
    map_idx = map_class(train_label_mtx, weight, mu)
    print_imag(map_idx, mu,mu_new, it)
    if np.sum(abs(mu_new - mu)) < 0.001 :
        converge_count = it
        mu = mu_new
        break
    #update mu
    mu = mu_new


<ipython-input-57-4303e905c4e1>:37: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Internal error in pre-inference rewriting pass encountered during compilation of function "print_imag" due to: Numba's print() function implementation does not support keyword arguments.

File "<ipython-input-57-4303e905c4e1>", line 47:
def print_imag(map_idx, mu, mu_new, iteration):
    <source elided>
        for (r,c) in product(range(28), range(28)):
            print(int(imagination_number[map_idx[label]][r][c]),end=" ")
            ^

  @nb.jit
<ipython-input-57-4303e905c4e1>:37: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "print_imag" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-57-4303e905c4e1>", line 44:
def print_imag(map_idx, mu, mu_new, iteration):
    <source elided>
    print("--------------------------------------

------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "print_imag" was compiled in object mode without forceobj=True.

File "<ipython-input-57-4303e905c4e1>", line 44:
def print_imag(map_idx, mu, mu_new, iteration):
    <source elided>
    print("------------------------------------------------------------")
    for label in range(10):
    ^

  state.func_ir.loc))
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-57-4303e905c4e1>", line 44:
def print_imag(map_idx, mu, mu_new, iteration):
    <source elided>
    print("------------------------------------------------------------")
    for label 

Streaming output truncated to the last 5000 lines.
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
labeled class  8 :
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 

In [56]:
def confusionMtrix( mu, train_img_mtx, train_label_mtx):
    CM = np.zeros((10,4), dtype = np.int32)   #TP, FN, FP, TN
    for label in range(10):
        for img in range(60000):
            true_label = train_label_mtx[img]
            prediction = map_idx[np.argmax(weight[img])]
            
            if map_idx[label]  == true_label:
                if prediction == label:
                    CM[map_idx[label]][0] += 1
                else:
                    CM[map_idx[label]][2] += 1
            else: 
                if prediction == label:
                    CM[map_idx[label]][1] += 1
                else:
                    CM[map_idx[label]][3] += 1
    return CM
CM = confusionMtrix(mu, train_img_mtx, train_label_mtx)

    
err = 0
for k in range(10):
    err += CM[k][1]
    Sensitivity = CM[k][0]/ (CM[k][0]+CM[k][1])
    Specificity = CM[k][3]/ (CM[k][2]+CM[k][3])
    print("")
    print("Confusion Matrix",k,":")
    print("\t\t","Predict number",k,"\tPredict not number",k)
    print("Is number",k,"\t\t",CM[k][0],"\t\t\t",CM[k][1])
    print("Isn't number",k,"\t\t",CM[k][2],"\t\t\t",CM[k][3])
    print("")
    print("Sensitivity (Successfully predict number ",k,")     :",Sensitivity)
    print("Specificity (Successfully predict not number ",k,") :",Specificity)
    print("")
    print("-----------------------------------------------------------")
print("Total iteration to converge: ",converge_count)
print("Total error rate: ",err / 60000)


Confusion Matrix 0 :
		 Predict number 0 	Predict not number 0
Is number 0 		 254 			 9473
Isn't number 0 		 5669 			 44604

Sensitivity (Successfully predict number  0 )     : 0.02611288166957952
Specificity (Successfully predict not number  0 ) : 0.8872356931155889

-----------------------------------------------------------

Confusion Matrix 1 :
		 Predict number 1 	Predict not number 1
Is number 1 		 389 			 14083
Isn't number 1 		 13095 			 92433

Sensitivity (Successfully predict number  1 )     : 0.026879491431730237
Specificity (Successfully predict not number  1 ) : 0.8759097111667046

-----------------------------------------------------------

Confusion Matrix 2 :
		 Predict number 2 	Predict not number 2
Is number 2 		 0 			 0
Isn't number 2 		 0 			 0

Sensitivity (Successfully predict number  2 )     : nan
Specificity (Successfully predict not number  2 ) : nan

-----------------------------------------------------------

Confusion Matrix 3 :
		 Predict number 3 	Predict

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in int_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in int_scalars
